# **PROYECTO 1 BI - DETECCIÓN DE FAKE NEWS**
---

Integrantes equipo #11:
*   Estudiante #1: Juan Pablo Barón - 202210502
*   Estudiante #2: María José Amorocho - 202220179
*   Estudiante #3: Julian Mondragón - 202221122

---

## **1. Entendimiento del negocio y enfoque analítico**

### 1.1. Generalidades

### 1.2. Impacto y enfoque analítico

---

## **2. Entendimiento y preparación de los datos**

### 2.1. Entendimiento de datos

### 2.2. Preparación de datos

---

## **3. Modelado y evaluación**

### 3.1. Estudiante 1 (Juan Pablo Barón): Modelo 1

### 3.2. Estudiante 2 (María José Amorocho): Modelo 2

### 3.3. Estudiante 3 (Julian Mondragón): Modelo 3

## **4. Resultados**

### 4.1. Descripción de resultados obtenidos y cómo aportan a lograr el objetivo del modelo planteado

### 4.2. Estrategia

### 4.3 Archivo de predicciones sobre los datos de prueba en formato CSV